In [40]:
import pandas as pd
import ast
from textblob import TextBlob
import re
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from langdetect import detect
from langcodes import Language

In [41]:
games = pd.read_csv('./games.csv', sep=',')
reviews = pd.read_csv('./reviews.csv', sep=',')
items = pd.read_csv('./items.csv', sep=',')

Clean EDA files are imported.

In [42]:
games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro


In [43]:
reviews.head(1)

,user_id,user_url,reviews
0,76561198047135310,http://steamcommunity.com/profiles/76561198047...,"[{'funny': '1 person found this review funny',..."


In [44]:
items.head(1)

,user_id,items_count,steam_id,user_url,items
0,WeiEDKrSat,28,76561198041660154,http://steamcommunity.com/id/WeiEDKrSat,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [45]:
bck_reviews = reviews.copy()

In [46]:
reviews = bck_reviews.copy()

## Sentiment_analysis

In [47]:
reviews['reviews'] = reviews['reviews'].apply(ast.literal_eval)

The <code>ast.literal_eval</code> function allows the reviews data to be taken as lists, since when imported from a csv they can be considered strings or other types that can cause errors when working with them.

In [48]:
def extract_item_id(review_list):
    item_id_list = []

    for review in review_list:
        if isinstance(review, dict) and 'item_id' in review:
            item_id_list.append(review['item_id'])
        else:
            item_id_list.append(None)

    return item_id_list

reviews['item_id'] = reviews['reviews'].apply(extract_item_id)

In [49]:
def extract_review(review_list):
    review_2_list = []

    for review in review_list:
        if isinstance(review, dict) and 'review' in review:
            review_2_list.append(review['review'])
        else:
            review_2_list.append(None)

    return review_2_list

reviews['review_2'] = reviews['reviews'].apply(extract_review)

The two functions (<code>extract_item_id</code> and <code>extract_review</code>) extract specific data found within the dictionaries of the review column and group them into a list in a new column.

In [50]:
def detect_language(text):
    try:
        lang_code = detect(text)
        lang_name = Language.get(lang_code).display_name()
        return lang_name
    except:
        return 'unknown'

In [51]:
reviews['language'] = reviews['review_2'].apply(lambda lst: detect_language(' '.join(lst)))

The <code>detect_language</code> function uses the import library Language to detect the language of the previously extracted reviews, because they are written in different languages.

In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
def preprocess_text(text: str, lang: str):
    text = text.lower()
    lang = lang.lower()
    
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    tokens = nltk.word_tokenize(text)
    
    try:
        stop_words = set(stopwords.words(lang))
    except:
        stop_words = set()
    
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens

In [54]:
reviews['review_clean'] = reviews.apply(lambda row: [preprocess_text(text, row['language']) for text in row['review_2']], axis=1)

The <code>preprocess_text</code> function converts all reviews to lowercase, and depending on the identified language uses a different set of stopwords. Then, it removes the stopwords detected in each review.

In [55]:
def sentiment_analysis(review_list):
    sentiment_list = []

    for inner_list in review_list:
        if len(inner_list) == 0:
            sentiment_list.append('1')
        else:
            total_sentiment = 0
            for text in inner_list:
                sentiment = TextBlob(text).sentiment.polarity

                if sentiment > 0:
                    total_sentiment += 1
                elif sentiment < -0.2:
                    total_sentiment -= 1

            if total_sentiment > 0:
                sentiment_list.append('2')
            elif total_sentiment < 0:
                sentiment_list.append('0')
            else:
                sentiment_list.append('1')

    return sentiment_list

In [56]:
reviews['sentiment_analysis'] = reviews['review_clean'].apply(sentiment_analysis)

The <code>sentiment_analysis</code> function analyzes each word of each review and assigns it a sentiment score according to the TextBlob library. Finally, it assigns 0 (Negative), 1 (Neutral) or 2 (Positive) depending on the final sentiment score of each review.

In [57]:
reviews = reviews.drop(columns=['review_2', 'review_clean', 'language'])

In [58]:
reviews.head()

,user_id,user_url,reviews,item_id,sentiment_analysis
0,76561198047135310,http://steamcommunity.com/profiles/76561198047...,"[{'funny': '1 person found this review funny',...","[224480, 220700]","[1, 1]"
1,hhaz,http://steamcommunity.com/id/hhaz,"[{'funny': '', 'posted': 'Posted February 13, ...","[211420, 241930]","[0, 2]"
2,D_55,http://steamcommunity.com/id/D_55,"[{'funny': '', 'posted': 'Posted March 31, 201...",[440],[2]
3,76561198072898248,http://steamcommunity.com/profiles/76561198072...,"[{'funny': '', 'posted': 'Posted December 6, 2...",[203160],[0]
4,76561198055370293,http://steamcommunity.com/profiles/76561198055...,"[{'funny': '', 'posted': 'Posted July 3, 2014....","[221100, 238320]","[1, 2]"


In [59]:
reviews.to_csv('./API_csv/sentiment_analysis_column.csv')

The dataframe with the new sentiment_analysis column is exported as a csv for use in creating the API.

## def user_data

In [60]:
items.head()

,user_id,items_count,steam_id,user_url,items
0,WeiEDKrSat,28,76561198041660154,http://steamcommunity.com/id/WeiEDKrSat,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,Leaf_Light_Moscow,5,76561198305694024,http://steamcommunity.com/id/Leaf_Light_Moscow,"[{'item_id': '4000', 'item_name': ""Garry's Mod..."
2,76561198061252210,25,76561198061252210,http://steamcommunity.com/profiles/76561198061...,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
3,Derp-e,18,76561198045022050,http://steamcommunity.com/id/Derp-e,"[{'item_id': '4000', 'item_name': ""Garry's Mod..."
4,sandwiches1,172,76561198052850911,http://steamcommunity.com/id/sandwiches1,"[{'item_id': '2400', 'item_name': 'The Ship', ..."


In [61]:
items['items'] = items['items'].apply(ast.literal_eval)
items['item_id'] = items['items'].apply(extract_item_id)

In [62]:
def extract_recommend(review_list):
    recommend_list = []

    for recommend in review_list:
        if isinstance(recommend, dict) and 'recommend' in recommend:
            recommend_list.append(recommend['recommend'])
        else:
            recommend_list.append(None)

    return recommend_list

reviews['recommend'] = reviews['reviews'].apply(extract_recommend)

In [63]:
def count_true(lst):
    return lst.count(True)

reviews['pct_recommend'] = reviews['recommend'].apply(lambda lst: lst.count(True) / len(lst) if len(lst) > 0 else 0)

In [64]:
items.head()

,user_id,items_count,steam_id,user_url,items,item_id
0,WeiEDKrSat,28,76561198041660154,http://steamcommunity.com/id/WeiEDKrSat,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 60, 70, 4000, 220, 340, 550, 223..."
1,Leaf_Light_Moscow,5,76561198305694024,http://steamcommunity.com/id/Leaf_Light_Moscow,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[4000, 221100, 304930, 323370, 346120]"
2,76561198061252210,25,76561198061252210,http://steamcommunity.com/profiles/76561198061...,"[{'item_id': '220', 'item_name': 'Half-Life 2'...","[220, 340, 4000, 12210, 550, 223530, 105600, 7..."
3,Derp-e,18,76561198045022050,http://steamcommunity.com/id/Derp-e,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[4000, 1250, 35420, 8980, 12210, 12220, 31280,..."
4,sandwiches1,172,76561198052850911,http://steamcommunity.com/id/sandwiches1,"[{'item_id': '2400', 'item_name': 'The Ship', ...","[2400, 2420, 2430, 3320, 240, 4000, 8000, 4560..."


In [65]:
reviews.head()

,user_id,user_url,reviews,item_id,sentiment_analysis,recommend,pct_recommend
0,76561198047135310,http://steamcommunity.com/profiles/76561198047...,"[{'funny': '1 person found this review funny',...","[224480, 220700]","[1, 1]","[True, True]",1.0
1,hhaz,http://steamcommunity.com/id/hhaz,"[{'funny': '', 'posted': 'Posted February 13, ...","[211420, 241930]","[0, 2]","[False, True]",0.5
2,D_55,http://steamcommunity.com/id/D_55,"[{'funny': '', 'posted': 'Posted March 31, 201...",[440],[2],[True],1.0
3,76561198072898248,http://steamcommunity.com/profiles/76561198072...,"[{'funny': '', 'posted': 'Posted December 6, 2...",[203160],[0],[True],1.0
4,76561198055370293,http://steamcommunity.com/profiles/76561198055...,"[{'funny': '', 'posted': 'Posted July 3, 2014....","[221100, 238320]","[1, 2]","[True, True]",1.0


In [66]:
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return value

games['price'] = games['price'].apply(convert_to_float)

In [67]:
def userdata(user_id: str):
    target_user_id = str(user_id)
    total_price = 0

    for index, row in items.iterrows():
        if target_user_id in row['user_id']:
            for item_id in row['item_id']:
                game_row = games[games['id']==float(item_id)]
                
                if not game_row.empty:
                    price = game_row['price'].values[0]
                    if price is not None:
                        if isinstance(price, (int, float)):
                            total_price += float(price)

    pct_recommend = reviews.loc[reviews['user_id'] == target_user_id, 'pct_recommend'].iloc[0]
    items_count = items.loc[items['user_id'] == target_user_id, 'items_count'].iloc[0]

    result_dict = {
                    'user': target_user_id,
                    'amount_spent': round(total_price, 2),
                    'recommendation_pct': pct_recommend,
                    'items_count': items_count
    }

    return result_dict

In [72]:
userdata("hhaz")

{'user': 'hhaz',
 'amount_spent': 948.41,
 'recommendation_pct': 0.5,
 'items_count': 85}

In [73]:
items.to_csv('./API_csv/userdata_items.csv')
reviews.to_csv('./API_csv/userdata_reviews.csv')
games.to_csv('./API_csv/userdata_games.csv')

## def countreviews

In [74]:
def extract_date(review_list):
    date_list = []

    for date in review_list:
        if isinstance(date, dict) and 'posted' in date:
            date_list.append(date['posted'])
        else:
            date_list.append(None)

    return date_list

reviews['date'] = reviews['reviews'].apply(extract_date)

In [75]:
def modify_date_format(date_string):
    match = re.search(r'(\w+ \d+, \d+)', date_string)
    if match:
        modified_date = match.group(1)
        return modified_date
    return date_string

reviews['date'] = reviews['date'].apply(lambda date_list: [modify_date_format(date) for date in date_list])

In [76]:
def convert_to_datetime(date_list):
    date_pattern = re.compile(r'\w+\s\d+,\s\d{4}')
    date_objects = []
    
    for date in date_list:
        try:
            date_objects.append(pd.to_datetime(date_pattern.search(date).group()).date())
        except AttributeError:
            date_objects.append(None) 
    
    return date_objects

reviews['date'] = reviews['date'].apply(convert_to_datetime)

In [77]:
reviews.head(1)

,user_id,user_url,reviews,item_id,sentiment_analysis,recommend,pct_recommend,date
0,76561198047135310,http://steamcommunity.com/profiles/76561198047...,"[{'funny': '1 person found this review funny',...","[224480, 220700]","[1, 1]","[True, True]",1.0,"[None, None]"


In [78]:
date_reviews = reviews.explode('date')

In [79]:
date_reviews['date'][0].dtype

dtype('O')

In [80]:
def countreviews(start_date: str, end_date: str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.strptime(end_date, "%Y-%m-%d").date()

    filtered_dates = date_reviews[(date_reviews['date'] >= start_date) & (date_reviews['date'] <= end_date)]
    unique_user_count = filtered_dates['user_id'].nunique()
    unique_pct_recommend = filtered_dates['pct_recommend'].mean()

    result_dict = {
                    'start_date': start_date,
                    'end_date': end_date,
                    'users_count': unique_user_count,
                    'recommend_pct': unique_pct_recommend
    }

    return result_dict

In [81]:
countreviews("2011-01-01", "2011-02-01")

{'start_date': datetime.date(2011, 1, 1),
 'end_date': datetime.date(2011, 2, 1),
 'users_count': 2,
 'recommend_pct': 1.0}

In [82]:
date_reviews.to_csv('./API_csv/countreviews_date_reviews.csv')

## def genre

In [83]:
def extract_playtime_forever(item_list):
    playtime_forever_list = []

    for item_dic in item_list:
        if isinstance(item_dic, dict) and 'playtime_forever' in item_dic:
            playtime_forever_list.append(item_dic['playtime_forever'])
        else:
            playtime_forever_list.append(None)

    return playtime_forever_list

items['playtime_forever'] = items['items'].apply(extract_playtime_forever)

In [84]:
items.head()

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,WeiEDKrSat,28,76561198041660154,http://steamcommunity.com/id/WeiEDKrSat,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 60, 70, 4000, 220, 340, 550, 223...","[328, 4, 42, 40, 1, 16441, 30, 0, 2142, 0, 134..."
1,Leaf_Light_Moscow,5,76561198305694024,http://steamcommunity.com/id/Leaf_Light_Moscow,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[4000, 221100, 304930, 323370, 346120]","[4548, 48, 1736, 390, 3]"
2,76561198061252210,25,76561198061252210,http://steamcommunity.com/profiles/76561198061...,"[{'item_id': '220', 'item_name': 'Half-Life 2'...","[220, 340, 4000, 12210, 550, 223530, 105600, 7...","[1250, 0, 10836, 17593, 7940, 0, 425, 41281, 2..."
3,Derp-e,18,76561198045022050,http://steamcommunity.com/id/Derp-e,"[{'item_id': '4000', 'item_name': 'Garry's Mod...","[4000, 1250, 35420, 8980, 12210, 12220, 31280,...","[174, 247, 0, 2155, 199, 0, 403, 77846, 29396,..."
4,sandwiches1,172,76561198052850911,http://steamcommunity.com/id/sandwiches1,"[{'item_id': '2400', 'item_name': 'The Ship', ...","[2400, 2420, 2430, 3320, 240, 4000, 8000, 4560...","[374, 0, 0, 639, 95, 4745, 0, 0, 0, 8, 0, 209,..."


In [85]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400.0,彼岸领域
4,No Info Available,['No Info Available'],Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,No Info Available,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,0.0,773570.0,No Info Available


In [86]:
genre_ranking = pd.DataFrame()
games = games.dropna()
items_playtime = items[['item_id', 'playtime_forever']].copy()
items_playtime = items_playtime.explode(['item_id', 'playtime_forever'])
items_playtime = items_playtime.dropna()
items_playtime['item_id'] = items_playtime['item_id'].astype(int)

In [87]:
items_playtime.head()

,item_id,playtime_forever
0,10,328
0,80,4
0,100,42
0,60,40
0,70,1


In [88]:
merged_data = pd.merge(games[['genres', 'id']], items_playtime, how='inner', left_on='id', right_on='item_id')

In [89]:
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

In [90]:
merged_data = merged_data.explode('genres')

In [91]:
merged_data = merged_data[['genres', 'playtime_forever']]
genre_rank = merged_data.groupby('genres')['playtime_forever'].sum().reset_index()
genre_rank['playtime_rank'] = genre_rank['playtime_forever'].rank(ascending=False).astype(int)
genre_rank = genre_rank.sort_values(by = 'playtime_rank', ascending=True)

In [92]:
genre_rank.head()

,genres,playtime_forever,playtime_rank
0,Action,147426176,1
9,Indie,77278723,2
13,RPG,48017415,3
15,Simulation,46247561,4
1,Adventure,43682074,5


In [93]:
def genre(genre:str):
    row = genre_rank[genre_rank['genres'].str.contains(genre, case=False, na=False)]
    
    if row.empty:
        return "Género no encontrado"
    
    playtime_rank = row.iloc[0]['playtime_rank']

    result_dict = {
                    'genre': genre,
                    'playtime_rank': playtime_rank
    }
    
    return result_dict

In [94]:
genre('Simulation')

{'genre': 'Simulation', 'playtime_rank': 4}

In [95]:
genre_rank.to_csv('./API_csv/genre_genre_rank.csv')

## def userforgenre

In [96]:
items.head(1)

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,WeiEDKrSat,28,76561198041660154,http://steamcommunity.com/id/WeiEDKrSat,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 60, 70, 4000, 220, 340, 550, 223...","[328, 4, 42, 40, 1, 16441, 30, 0, 2142, 0, 134..."


In [97]:
items_user_playtime = items[['item_id', 'playtime_forever', 'user_id', 'user_url']].copy()
items_user_playtime = items_user_playtime.explode(['item_id', 'playtime_forever'])
items_user_playtime = items_user_playtime.dropna()
items_user_playtime['item_id'] = items_playtime['item_id'].astype(int)

In [99]:
id_genres = games[['id', 'genres']].copy()
id_genres['genres'] = id_genres['genres'].apply(ast.literal_eval)
id_genres = id_genres.explode('genres')
id_genres['id'] = id_genres['id'].astype(int)

In [100]:
id_genres.head()

,id,genres
0,761140,Action
0,761140,Casual
0,761140,Indie
0,761140,Simulation
0,761140,Strategy


In [101]:
merged_genres_playtime = pd.merge(id_genres, items_user_playtime, how='inner', left_on='id', right_on='item_id')

In [102]:
merged_genres_playtime.head()

,id,genres,item_id,playtime_forever,user_id,user_url
0,282010,Action,282010,232,phrostb,http://steamcommunity.com/id/phrostb
1,282010,Action,282010,16,tzarius,http://steamcommunity.com/id/tzarius
2,282010,Action,282010,2,76561198003546877,http://steamcommunity.com/profiles/76561198003...
3,282010,Action,282010,102,DakiniBrave,http://steamcommunity.com/id/DakiniBrave
4,282010,Action,282010,19,devoid,http://steamcommunity.com/id/devoid


In [103]:
playtime_rank = merged_genres_playtime.groupby(['genres', 'user_id', 'user_url'])['playtime_forever'].sum().reset_index()

In [104]:
playtime_rank.sort_values(by=['genres', 'playtime_forever'], ascending=[True, False], inplace=True)
playtime_rank['rank'] = playtime_rank.groupby('genres').cumcount() + 1

In [105]:
playtime_rank.head()

,genres,user_id,user_url,playtime_forever,rank
1468,Action,SuNNZ,http://steamcommunity.com/id/SuNNZ,589348,1
1461,Action,Steamified,http://steamcommunity.com/id/Steamified,574446,2
2024,Action,phrostb,http://steamcommunity.com/id/phrostb,474268,3
1713,Action,de_reyals,http://steamcommunity.com/id/de_reyals,461222,4
775,Action,76561198086282677,http://steamcommunity.com/profiles/76561198086...,384055,5


In [106]:
def userforgenre(genre: str):
    genre_data = playtime_rank[playtime_rank['genres'] == genre].head(5)

    result_dict = genre_data.to_dict(orient='records')

    return result_dict

In [107]:
userforgenre('Action')

[{'genres': 'Action',
  'user_id': 'SuNNZ',
  'user_url': 'http://steamcommunity.com/id/SuNNZ',
  'playtime_forever': 589348,
  'rank': 1},
 {'genres': 'Action',
  'user_id': 'Steamified',
  'user_url': 'http://steamcommunity.com/id/Steamified',
  'playtime_forever': 574446,
  'rank': 2},
 {'genres': 'Action',
  'user_id': 'phrostb',
  'user_url': 'http://steamcommunity.com/id/phrostb',
  'playtime_forever': 474268,
  'rank': 3},
 {'genres': 'Action',
  'user_id': 'de_reyals',
  'user_url': 'http://steamcommunity.com/id/de_reyals',
  'playtime_forever': 461222,
  'rank': 4},
 {'genres': 'Action',
  'user_id': '76561198086282677',
  'user_url': 'http://steamcommunity.com/profiles/76561198086282677',
  'playtime_forever': 384055,
  'rank': 5}]

In [108]:
playtime_rank.to_csv('./API_csv/userforgenre_playtime_rank.csv')

## def developer

In [109]:
developer_year_price = games[['release_date', 'price', 'developer']].copy()
developer_year_price['release_date'] = developer_year_price['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
non_numeric_rows = developer_year_price.loc[pd.to_numeric(developer_year_price['price'], errors='coerce').isna()]

In [110]:
grouped_developer_no_numeric = non_numeric_rows.groupby(['developer', 'release_date'])['price'].count().reset_index()
grouped_developer_no_numeric = grouped_developer_no_numeric.sort_values(by = ['developer', 'release_date'], ascending=True)

grouped_developer = developer_year_price.groupby(['developer', 'release_date'])['price'].size().reset_index()
grouped_developer = grouped_developer.sort_values(by = ['developer', 'release_date'], ascending=True)

grouped_developer_no_numeric = grouped_developer_no_numeric.rename(columns={'price': 'count_free'})
grouped_developer = grouped_developer.rename(columns={'price': 'total_count'})

In [111]:
merged_developer = pd.merge(grouped_developer, grouped_developer_no_numeric, how='left', on=['developer', 'release_date'])
merged_developer = merged_developer.rename(columns={'release_date': 'year'})
merged_developer['pct_free'] = merged_developer['count_free'] / merged_developer['total_count']
merged_developer['pct_free'].fillna(0, inplace=True)

In [112]:
def developer(developer: str):
    result_df = merged_developer[merged_developer['developer'] == developer].reset_index()

    result_dict = result_df[['year', 'pct_free']].to_dict(orient='records')
    return result_dict

In [113]:
developer('Activision')

[{'year': 1993, 'pct_free': 0.0},
 {'year': 1996, 'pct_free': 0.0},
 {'year': 1997, 'pct_free': 0.0},
 {'year': 2000, 'pct_free': 0.0}]

In [114]:
merged_developer.to_csv('./API_csv/developer_merged_developer.csv')

## def sentiment_analysis

In [115]:
reviews_sentiment = reviews[['user_id','item_id', 'sentiment_analysis']].copy()
reviews_sentiment = reviews_sentiment.explode(['item_id','sentiment_analysis'])
reviews_sentiment['item_id'].fillna(0, inplace=True)
reviews_sentiment['item_id'] = reviews_sentiment['item_id'].astype(int)

In [116]:
sentiment_analysis_developer = pd.merge(reviews_sentiment, games[['id','developer', 'release_date']], how='inner', left_on='item_id', right_on='id')
sentiment_analysis_developer['release_date'] = sentiment_analysis_developer['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
sentiment_analysis_developer = sentiment_analysis_developer.drop(columns=['user_id', 'item_id', 'id'])
sentiment_analysis_developer = sentiment_analysis_developer.sort_values(by = ['developer', 'release_date'], ascending=[True, True])

In [117]:
sentiment_analysis_developer.head()

,sentiment_analysis,developer,release_date
4674,1,11 bit studios,2011
4353,0,11 bit studios,2014
4354,1,11 bit studios,2014
4355,2,11 bit studios,2014
4582,2,1C Company,2008


In [118]:
def sentiment_analysis(year: int):
    
    year_df = sentiment_analysis_developer[sentiment_analysis_developer['release_date'] == year]
    
    counts = year_df['sentiment_analysis'].value_counts().to_dict()
    
    for sentiment in [0, 1, 2]:
        if sentiment not in counts:
            counts[sentiment] = 0
    
    sentiment_dict = {
        'Negative': counts['0'],
        'Neutral': counts['1'],
        'Positive': counts['2']
    }
    
    return sentiment_dict

In [119]:
sentiment_analysis(2011)

{'Negative': 50, 'Neutral': 158, 'Positive': 154}

In [120]:
sentiment_analysis_developer.to_csv('./API_csv/sentiment_analysis_sentiment_analysis_developer.csv')